In [ ]:
import numpy as np
from scipy import spatial

In [ ]:
class Racetrack:
    '''
    Racetrack contains ordered lists of waypoints denoting the DeepRacer racetrack interior boundary, exterior boundary, and centerline
    ---
    Member variables
    ---
    waypointFilePath: (string) path and filename denoting the DeepRacer racetrack file data in numpy format
    all_waypoints: (ndarray) of waypoint coordinates in a 6-tuple
    waypoints_center: (ndarray) of waypoint coordinates of centerline
    waypoints_inner: (ndarray) of waypoint coordinates of inner track boundary
    waypoints_outer: (ndarray) of waypoint coordinates of outer track boundary
    waypoints_inner_angles: (ndarray) absolute angles of inner boundary waypoints
    waypoints_inner_distances: (ndarray) distances between inner boundary waypoints
    waypoints_inner_kdtree: (kdtree) contains waypoint coordinates for efficient lookup of nearest inner track boundary waypoint
    waypoints_outer_kdtree: (kdtree) contains waypoint coordinates for efficient lookup of nearest outer track boundary waypoint
    trackwidth: width of the racetrack calulated as the distance between the first two interior boundary waypoints
    '''
    def __init__(self, waypointFilePath):
        '''
        waypointFilePath: (string) path denoting the track file in numpy format
        '''
        self.waypointFilePath = waypointFilePath
        self.all_waypoints = np.load(waypointFilePath)
        #Remove duplicate waypoints
        self.all_waypoints, aw_indices = np.unique(self.all_waypoints, axis=0, return_index=True)
        self.all_waypoints = self.all_waypoints[np.argsort(aw_indices)]
        [self.waypoints_center, self.waypoints_inner, self.waypoints_outer] = np.split(self.all_waypoints, 3, axis=1)
        self.waypoints_inner_kdtree = spatial.KDTree(self.waypoints_inner)
        self.waypoints_outer_kdtree = spatial.KDTree(self.waypoints_outer)
        #Calculate trackwidth based on inner waypoint path index 0
        [self.trackwidth, outer_min_idx] = self.waypoints_inner_kdtree.query(self.waypoints_outer[0])
        #Calculate inner angles and distances
        self.waypoints_inner_angles = []
        self.waypoints_inner_distances = []
        [self.waypoints_inner_angles, self.waypoints_inner_steering_angles, self.waypoints_inner_distances] = self.calculateWaypointDistancesAndAngles(self.waypoints_inner)
        [self.waypoints_center_angles, self.waypoints_center_steering_angles, self.waypoints_center_distances] = self.calculateWaypointDistancesAndAngles(self.waypoints_center)

        
    def calculateWaypointDistancesAndAngles(self, waypoint_arr, denormalize_angles=False):
        '''
        Calculates three arrays based on the input waypoint_arr
        Returns
            absolute_angles_arr: the absolute (origin-relative) angles of each waypoint
            steering_angles_arr: the change in angle to the next waypoint relative to the prior waypoint
            distances_arr: the euclidean distance from the prior waypoint
        ---
        waypoint_arr: (list of coordinate array [(float) x,(float) y])
        denormalize_angles = if True, then convert angles from [-360, 360] space to [0, ~], suitable for continuous plotting
        '''
        absolute_angles_arr = []
        steering_angles_arr = []
        distances_arr = []
        for idx, waypoint in enumerate(waypoint_arr):
            prev_waypoint = waypoint_arr[idx-1]
            prev_prev_waypoint = waypoint_arr[idx-2]
            [angle, distance, relative_steering_angle] = self.angleAndDistanceBetweenWaypoints(waypoint, prev_waypoint, prev_prev_waypoint)
            
            if denormalize_angles:
                if(relative_steering_angle > 180): 
                    steering_angles_arr.append(relative_steering_angle - 360)
                elif(relative_steering_angle < -180): 
                    steering_angles_arr.append(relative_steering_angle + 360)
                else:
                    steering_angles_arr.append(relative_steering_angle)
                #Absolute angle; skip the first 
                if idx == 0:
                    absolute_angles_arr.append(0)
                else: absolute_angles_arr.append(absolute_angles_arr[idx-1] + steering_angles_arr[idx])
            else:
                angle = angle if angle >= 0 else (angle + 360)
                absolute_angles_arr.append(angle)
                steering_angles_arr.append(relative_steering_angle)
                distances_arr.append(distance)
                
        if denormalize_angles:
            steering_angles_arr.pop(0)
            #absolute_angles_arr.pop(0)  
            
        return absolute_angles_arr, steering_angles_arr, distances_arr
        
    def distanceToTrackEdges(self, p):
        '''
        p: a list of [x, y] coordinates from which to determine the distance to track edges
        returns inner_dist, outer_dist, inner_min_idx, outer_min_idx where negative distances indicate outside the track boundaries
        '''
        [inner_dist, inner_min_idx] = self.waypoints_inner_kdtree.query(p)
        [outer_dist, outer_min_idx] = self.waypoints_outer_kdtree.query(p)
        track_width = np.linalg.norm(self.waypoints_inner[inner_min_idx] - self.waypoints_outer[outer_min_idx])
        if(inner_dist > outer_dist and inner_dist > track_width): outer_dist *= -1
        if(inner_dist < outer_dist and outer_dist > track_width): inner_dist *= -1
        return inner_dist, outer_dist, inner_min_idx, outer_min_idx
    
    #Function to help plot waypoints
    def plotWaypointSeries(self, waypoint_arr, hex_color):
        '''
        Generates a scatter plot and line segment series to map a set of waypoints
        ---
        waypoint_arr: (list of points [(float64) x,(float64) y]) the waypoints to plot
        hex_color: the hex color to shade waypoints and line segments
        '''
        [waypoints_x, waypoints_y] = np.split(waypoint_arr, 2, axis=1);
        plt.scatter(waypoints_x, waypoints_y, c=hex_color)
        plt.plot(waypoints_x, waypoints_y, c=hex_color)
        
    #Function to help plot track boundaries
    def plotTrackSeries(self, hex_color='#17becf'):
        '''
        Plot track boundaries
        ---
        hex_color: (string) the hex color to apply to boundary waypoints
        '''
        self.plotWaypointSeries(self.waypoints_inner, hex_color)
        self.plotWaypointSeries(self.waypoints_outer, hex_color)
        
        
    #Get the absolute angles and distance between two waypoints
    def angleAndDistanceBetweenWaypoints(self, a, b, c=[]):
        '''
        Function to calculate three quanties
        returns
            absolute (origin-relative) angle between waypoint a and b
            distance between waypoint a and b
            relative angle between waypoint a and b as delta from waypoint c and a. Only calculated if waypoint c is provided
        ---
        a: (point [(float64 x, float64 y)]) start waypoint
        b: (point [(float64 x, float64 y)]) destination waypoint
        c: (optional point [(float64 x, float64 y)]) waypoint prior to start waypoint. If this is included, then the relative steering angle between a and b will also be calculated and returned
        '''
        angle = math.atan2(b[1] - a[1], b[0] - a[0]) * 180 / math.pi
        distance = np.linalg.norm(b - a)
        relative_steering_angle = None
        if len(c) > 0:
            prior_angle = math.atan2(a[1] - c[1], a[0] - c[0]) * 180 / math.pi
            relative_steering_angle = angle - prior_angle + 180
            if relative_steering_angle > 360: relative_steering_angle -= 360
            if relative_steering_angle < -360: relative_steering_angle += 360
        return angle, distance, relative_steering_angle
    
    #Get the maximum velocity at a given steering angle in m/s
    def maxVelocityAtSteeringAngle(self, angle):
        '''
        Note that this is a linear placeholder which is almost certainly completely wrong!
        4 m/s at 0 degrees
        ~1 m/s at 30 degrees 
        minimum 1 m/s
        '''
        #velocity = 4 + (angle * (-3/10))
        velocity = (2 / (.66 + angle)) + 1
        velocity = 1 if velocity < 1 else velocity
        return velocity